# Práctica 2: Manejo básico de ROS2

## Objetivo

Que el alumno haga uso de las funciones básicas de ROS2 (publicador y suscriptor) y de sus aplicaciones con turtlesim y rviz2

### Metas

Que el alumno:
- Integre texto en Markdown dentro de Jupyter
- Integre código de Python dentro de Jupyter
- Haga uso de la terminal de Ubuntu y contruya un script de bash
- Genere un repositorio en GitHub y comparta su archivo de práctica

### Contribución al perfil del egresado

La siguiente práctica contribuye en los siguientes puntos al perfil del egresado:

#### Aptitudes y habilidades

- Para diseñar, construir, operar y mantener los sistemas mecatrónicos y sus componentes.
- Para crear, innovar o evaluar las tecnologías relacionadas con la mecatrónica.

#### Actitudes profesionales

- Ser creativo e innovador.
- Tener confianza en su preparación académica.
- Comprometido con su actualización, superación y competencia profesional.

#### Actitudes de tipo social

- Promover el cambio en la mentalidad frente a la competitividad internacional.

## Rúbrica de evaluación

### La evaluación de la práctica contará de los siguientes puntos

| Elemento | Porcentaje |
| ------:| ----------- |
| **Cuestionario previo** | 15% | 
| **Desarrollo** | 35% |
| **Análisis de resultados**  | 35% |
| **Conclusiones** | 15% |

### Se evaluará con los siguientes criterios:

| Elemento | Malo | Regular | Bueno |
| ------:| ------ | --------| ------|
| **Cuestionario previo** | El trabajo no contiene cuestionario previo o todas las preguntas son incorrectas (0%)| Al menos la mitad de las preguntas son correctas (8%) |  Todas las preguntas son correctas (15%) |
| **Desarrollo** | El trabajo no contiene desarrollo o su planteamiento no concuerda con lo deseado (0%) | El desarrollo está mal planteado o no llega a los resultados esperados (10%) | El desarrollo tiene un planteamiento adecuado y llega a los resultados esperados (35%) |
| **Análisis de resultados**  | El trabajo no contiene análisis de resultados o la información no se está interpretando correctamente (0%) | La interpretación de los resultados es parcial o desorganizada (10%) | Realiza un correcto análisis de los resultados de forma organizada   (35%) |
| **Conclusiones** | El trabajo no contiene conclusiones o no hacen referencia al trabajo desarrollado y los objetivos planteados (0%) | La redacción de las conclusiones es desorganizada o confusa (8%) | Las conclusiones del trabajo son claras y hacen referencia al trabajo desarrollado y los objetivos planteados (15%) | 

## Introducción

### ROS2

ROS 2 (Robot Operating System 2) es un conjunto de bibliotecas y herramientas que permiten desarrollar software para robots de forma modular, distribuida y escalable.
En ROS 2, los programas se comunican entre sí mediante mensajes que viajan sobre una red (incluso entre computadoras distintas), usando un sistema de comunicación llamado DDS (Data Distribution Service).

<img src="imagenes/ROS2_logo.webp" alt = "ROS2" height="100" display= "block"/>

#### Nodos

Un nodo es la unidad básica de ejecución en ROS 2.
Cada nodo es un programa que realiza una tarea específica dentro del sistema del robot: leer un sensor, controlar un motor, procesar imágenes, etc.
En Python, los nodos se definen a partir de la clase base rclpy.node.Node.
~~~python
from rclpy.node import Node

class Nodo(Node):
    def __init__(self):
        super().__init__('nombre_nodo')
        self.get_logger().info("Nodo iniciado")
~~~

#### Tópicos

Los tópicos (topics) son canales de comunicación asíncrona y continua entre nodos.
Un nodo publica mensajes en un tópico, y otros nodos se suscriben para recibirlos.
Todos los mensajes que viajan por un tópico tienen un tipo de dato definido, por ejemplo:
- std_msgs/msg/String
- geometry_msgs/msg/Twist

Por ejemplo:

- Un nodo A (sensor) publica temperaturas en el tópico /temperatura.
- Otro nodo B (monitor) se suscribe a /temperatura y muestra los valores.

Para generar un publicador, se crea un publicador definiendo el tipo de mensaje y el tópico por el que se publica. Después se manda a llamar al publicador para mandar un mensaje.
~~~python
  topic_name = "/topic"
  self.publisher = self.create_publisher(String, topic_name, 10)

  msg = String()
  msg.data = "Message"
  self.publisher.publish(msg)
~~~
Para generar un subscriptor:
~~~python 
  topic_name = "/topic"
  self.subscription = self.create_subscription(
    String, topic_name, self.listener_callback, 10)
~~~

#### Servicios

Los servicios (services) son una forma de comunicación sincrónica (de solicitud–respuesta). Funcionan como una llamada a función entre nodos: uno envía una solicitud (request), y el otro responde con un resultado (response).

Los servicios se usan para operaciones puntuales, no continuas (por ejemplo, mover un robot a una posición, tomar una foto, guardar datos, etc.).

Por ejemplo:

- Un nodo A (cliente) pide: “calcula la suma de 2 y 3”.
- Otro nodo B (servidor) responde: “la suma es 5”.

Para crear un servidor, hay que crear el objeto y la función que se llama al recibir una solicitud

~~~python
  self.srv = self.create_service(AddTwoInts, server_name, self.add_two_ints_callback)
def add_two_ints_callback(self, request:AddTwoInts.Request, response:AddTwoInts.Response):
  response.sum = request.a + request.b
  self.get_logger().info("Recibido: a={}, b={}, respuesta={}".format(request.a, request.b, response.sum))
  return response
~~~

Para crear un cliente, hay que crear el objeto del cliente y la solicitud, enviarla, y definir la función que se llamará al recibir el objeto de la respuesta
~~~python
self.client = self.create_client(AddTwoInts, service_name)
# Esperamos al servicio
while not self.client.wait_for_service(timeout_sec=1.0):
  self.get_logger().info('Esperando al servicio {}...'.format(service_name))
request = AddTwoInts.Request()
request.a = self.a
request.b = self.b
# --- Manda solicitud
future = self.client.call_async(request)
# --- Función que se ejecuta al recibir respuesta
future.add_done_callback(self.callback_result)
~~~

#### Parámetros

Los parámetros son valores configurables que un nodo puede leer o modificar durante su ejecución.
Permiten ajustar el comportamiento del nodo sin cambiar el código.

Por ejemplo:

- Un nodo de control de motores puede tener un parámetro max_velocity de 0.1.
- Otro nodo puede cambiarlo a 0.5 sin interrumpir al primer nodo.

Por ejemplo, si en un nodo queremos que el tópico utilizado sea un parámetro
~~~python
self.declare_parameter("topic_param", "/topic")
topic_name = self.get_parameter("topic_param").value
~~~

Una forma gráfica de ver las comunicaciones en ROS2:

<img src="imagenes/Nodes-TopicandService.gif" height = "400" display = "block" alt = "Vosualización de las comunicaciones en ROS2"/>

### URDF (Unified Robot Description Format):

El URDF es un formato de archivo XML que se utiliza para describir la estructura de un robot, comunmente usado en ROS. Permite definir la composición física de un robot, incluyendo sus eslabones y juntas. Además, permite especificar características como el peso, la geometría, y la disposición espacial de los elementos del robot.

Un archivo URDF básico se construye a partir de **eslabones** y **juntas**.

#### Para definir un eslabón, se utiliza la estructura siguiente:
~~~xml
<!--Definición del eslabón-->
<link name = "nombre_del_eslabón">
  <!--
  Elementos visuales del eslabón 
  (modelos o formas geométricas)
  -->
  <visual>
      <!--
      Posición y rotación del elemento visual
      respecto al origen del eslabón
      -->
      <origin xyz = "x y z" rpy = "gamma beta alfa"/> 
        <!--Elementos visuales asociados a la junta-->
        <geometry> 
        <!--Primitiva visual (caja en este caso)-->
        <box size = "x y z"/> 
     </geometry>
        <!--Material usado para el elemento visual-->
        <material name = "nombre_del_material"> 
          <color rgba = "r g b a"/> 
        </material> 
  </visual> 
</link> 
~~~

Los elementos primitivos son formas geométricas básicas que se utilizan para representar las partes de un robot en URDF. Algunos de los más comunes son:

- **Caja (box)**: Un paralelepípedo definido por su ancho, alto y profundidad.
>~~~xml
>  <box size = "x y z"/> 
>~~~
- **Cilindro (cylinder)**: Definido por su radio y longitud.
>~~~xml
>  <cylinder radius = "r" length="l"/> 
>~~~
- **Esfera (sphere)**: Definida por su radio.
>~~~xml
>  <sphere radius = "r"/> 
>~~~
- **Malla (mesh)**: Permite importar geometrías más complejas en formatos como STL, Collada, obj, entre otros.
>~~~xml
>  <mesh filename = "ruta/al/archivo"/> 
>~~~

Se puede agregar cómo se desplegarán estos elementos a través de un **material**: 
- **Color**: Color plano con el que se despliega el elemento, definido por componentes rgb y transparencia (alfa).
>~~~xml
> <material name = "Nombre"> 
>   <color rgba = "r g b a"/> 
> </material> 
>~~~
- **Textura**: Imagen que se despliega sobre el modelo.
>~~~xml
> <material name = "Nombre"> 
>   <texture filename = "ruta/al/archivo"/> 
> </material> 
>~~~

#### Para una junta: 
~~~xml
<!--Definición de la junta-->
<joint name = "nombre_de_la_junta" type = "tipo_de_junta"> 
  <!--
  Eslabones padre e hijo
  - Padre: Eslabón respecto al cual se mide la rotació o traslación
  - Hijo: Eslabón que se desplaza o rota
  -->
  <parent link = "eslabon_padre_(fijo)"/> 
  <child link = "eslabon_hijo_(movible)"/> 
  <origin xyz = "x y z" rpy = "gamma beta alfa"/> 
  <!--Eje de rotación  (1: se mueve. 0: no se mueve)-->
  <axis xyz = "x y z"/> 
  <!--Límites 
  - effort: par máximo
  - velocity: velocidad absoluta máxima
  - lower-upper: posición mínima y máxima-->
  <limit effort="10.0" lower="-3.14" upper="3.14" velocity="3.14"/>
</joint>
~~~
En el formato URDF, existen varios tipos de juntas que pueden definir el movimiento relativo entre las partes de un robot:

- **Junta fija**: No permite ningún movimiento entre los elementos conectados.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "fixed"> 
>~~~
- **Junta rotacional**: Permite un movimiento de rotación alrededor de un eje.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "revolute"> 
>~~~
- **Junta prismática**: Permite un movimiento lineal a lo largo de un eje.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "prismatic"> 
>~~~
- **Junta continua**: Es una variación de la junta rotacional que permite una rotación sin límites.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "continuous"> 
>~~~

### RVIZ


RViz (ROS Visualization) es una herramienta gráfica dentro del ecosistema de ROS que permite visualizar una amplia gama de información generada por robots. Esta herramienta es esencial para depurar, entender y visualizar los datos que se generan en tiempo real durante la simulación o la operación de un robot en un entorno físico.

<img src="imagenes/rviz.png" alt = "RViz" height="300" display= "block"/>


RViz soporta muchos tipos de visualizaciones, en este caso veremos:

- **Modelos 3D del robot (URDF)**: Puedes cargar y visualizar la descripción del robot usando su archivo URDF (Unified Robot Description Format). Esto te permite ver un modelo 3D del robot y observar cómo se mueve cada parte del robot (es decir, los eslabones y las juntas). También puedes ver cómo se actualiza en tiempo real cuando el robot cambia de posición.

- **Transformaciones (TF)**: ROS utiliza un sistema de coordenadas llamado TF (transformación de marcos) para definir la relación espacial entre diferentes partes del robot y el entorno. En RViz, puedes visualizar estos sistemas de referencia y las transformaciones entre ellos, lo que es útil para entender cómo se mueven y orientan los diferentes componentes del robot.
Esto incluye visualizar el origen de coordenadas del robot y sus partes móviles.

## Cuestionario previo

### Responder de forma breve las siguientes preguntas:

- ¿Qué es un ejecutor (executor) en ROS2?
> Es un componente el cual se encarga de gestionar y administrar las devoluciones de callbacks de los nodos en el grafo de comunicación, por ejemplo, procesando eventos como la llegada de mensajes en suscriptores, la expiración de temporizadores o el manejo de servicios y clientes. Básicamente el ejecutor toma los nodos registrados y ejecuta sus callbacks de manera concurrente o secuencial. Existen diferentes tipos donde destacan 3: SingleThreadedExecutor: este ejecuta todo en un solo hilo, lo cual evita errores de concurrencia; MultiThreadedExecutor: valiéndose de múltiples hilos para procesar callbacks en paralelo, idóneo para aplicaciones con alta carga I/O; StaticSingleThreadedExecutor: parecido al primero con la diferencia de que optimiza los nodos conocidos de antemano.

- ¿Qué es la ejecución por hilos en python?
> Es el uso del módulo threading para crear y gestionar hilos independiente dentro de un proceso, en este caso un hilo se define como la unidad de ejecución ligera que comparte el mismo espacio de memoria y recursos del proceso principal, permitiendo la concurrencia (sin ser paralelas realmente). Donde el hilo principal se define como el hilo por defecto donde se ejecuta el hilo código inicial; por otra parte el hilo secundario se entiende como aquellos que pueden ejecutarse de forma concurrente con el principal. Ahora bien este tiene ciertas limitaciones las cuales la delimitan a un uso ideal en tareas de I/O-bound o para evitar bloqueos en interfaces gráficas.

- ¿Qué ventajas tiene ejecutar un programa en un hilo secundario?
> Ejecutar un programa o una parte parcial del mismo en un hilo secundario tiene como ventajas principales: no bloquear el principal con lo cual puede continuar ejecutando tareas y con ello evitando que el programa entero se “congele” durante operaciones largas como esperas de red o I/; concurrencias para tareas I/O-bound con lo cual la hacen ideal para operaciones como lecturas de sensores en ROS (suscriptores) donde el hilo espera datos sin consumir recursos de la CPU; mejora la responsividad ya que en aplicaciones interactivas permite que el usuario siga interactuando mientras un hilo maneja simulaciones URDF o bucles ROS; escalabilidad ya que facilita manejar múltiples nodos ROS concurrentes (uno para publicación y otro para suscripcion) sin secuencializar todo; facilita la depuración y modularidad dado que separa la lógica em hilos para executor ROS y otro para visualización reduciendo complejidad y facilitando pruebas.

- ¿Qué pasaría si se utiliza un comando que se queda en periodo de espera (como rclpy.spin()) dentro de un entorno de celdas como jupyter?
> Al ejecutarse ocasiona un bloqueo del kernel ya que la celda se “colgara” indefinidamente dado que <spin()> entra en un bucle infinito que solo se detiene con <rclpy.shutdown()> lo una interrupción externa, el kernel de Júpiter no puede avanzar a otras celdas ni responder a inputs hasta que este se libere. Además de que se tiene una pérdida de interactividad pues no se podrá ejecutar más código en ese notebook sin interrumpir el kernel con lo cual se causan timeout o crash si no se se maneja aunado esto tiene un impacto en ROS pues los nodos creados en esa celada procesaron mensajes temporales pero el notebook entero quedaría inutilizable ya que se rompieron flujos interactivos como depuración de URDF o visualización en tiempo real.


*En caso de integrar imagenes, colocarlas en la carpeta "imagenes"*

## Desarrollo

### 1. Tópicos en ROS2

En este primera parte, se usarán las funciones básicas de ROS2 dentro de jupyter

Para ello, en celdas separadas se realizarán las siguientes acciones:

- Inicializar ros2 e importar dependencias (nodos, mensajes, ejecución por hilos y ejecutor de ros2)
~~~python
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
import threading
from rclpy.executors import MultiThreadedExecutor
~~~
- Crear una clase que herede de nodo, que implemente un publicador y sus funciones
~~~python
  class Publisher(Node):
    def __init__(self):
      # Constructor de inicio
    def timer_callback(self):
      # Función de timer para publicar periódicamente cada segundo
~~~
- Crear una clase que herede de nodo, que implemente un subscriptor y sus funciones
~~~python
  class Subscriber(Node):
    def __init__(self):
      # Constructor
    def listener_callback(self, msg):
      # Función que se llama al recibir un mensaje
~~~
- Verificar si ROS2 está inicializado. Si no, inicializarlo. Utilizar la función rclpy.ok(), que nos devuelve el estado de ROS2
~~~python
  if not rclpy.ok():
    rclpy.init(args=None)
~~~
- Instanciar un ejecutor de ROS2 que se encargue de varios nodos de forma simultánea, y ejecutarlo en segundo plano para no bloquear la terminal
~~~ python
  executor = MultiThreadedExecutor()
  thread = threading.Thread(target=executor.spin, daemon=True)
  thread.start()
~~~
- Instanciar los nodos publicador y suscriptor y agregarlos al ejecutor
~~~python
  pub_node = SimplePublisher()
  sub_node = SimpleSubscriber()
  executor.add_node(sub_node)
  executor.add_node(pub_node)
~~~
- Destruir los nodos para que dejen de ejecutarse
~~~python
  pub_node.destroy_node()
  sub_node.destroy_node()
~~~
- Que detenga el proceso de ROS2 en caso de que siga activo (para evitar procesos en segundo plano)
~~~python
  if rclpy.ok():
    rclpy.shutdown()
~~~

Como probatorio: 

- Las celdas con código en esta sección del archivo

In [8]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
import threading
from rclpy.executors import MultiThreadedExecutor

In [9]:
class SimplePublisher(Node):
  def __init__(self):
    super().__init__('Simple_publisher')
    topic_name = "/topic"
    self.publisher = self.create_publisher(String, topic_name, 10)
    self.timer = self.create_timer(1.0, self.timer_callback)
    self.get_logger().info('Publicador iniciado en {}'.format(topic_name))
    self.count = 0
  def timer_callback(self):
    msg = String()
    msg.data = "Mensaje # {}".format(self.count)
    self.publisher.publish(msg)
    self.get_logger().info('Publicando: {}'.format(msg.data))
    self.count += 1

class SimpleSubscriber(Node):
  def __init__(self):
    super().__init__('Simple_subscriber')
    topic_name = "/topic"
    self.subscription = self.create_subscription(String, topic_name, self.listener_callback, 10)
    self.get_logger().info('Suscriptor escuchando tópico: {}'.format(topic_name))
  def listener_callback(self, msg):
    self.get_logger().info(f'Recibido: "{msg.data}"')

In [10]:
if not rclpy.ok():
  rclpy.init(args=None)

In [11]:
executor = MultiThreadedExecutor()
thread = threading.Thread(target = executor.spin, daemon = True)
thread.start()

In [ ]:
pub_node = SimplePublisher()
sub_node = SimpleSubscriber()
executor.add_node(sub_node)
executor.add_node(pub_node)

In [14]:
pub_node.destroy_node()
sub_node.destroy_node()

if rclpy.ok():
  rclpy.shutdown()

- Dejar los mensajes de salida que se imprimirán al agregar los nodos al ejecutor

<img src="imagenes/Probatorio1.png" height=500/>

### 2. Servicios en ROS2

En este segunda parte, se usarán los servicios de ROS2 dentro de jupyter

Para ello, en celdas separadas se realizarán las siguientes acciones:

- Inicializar ros2 e importar dependencias (nodos, mensajes, ejecución por hilos y ejecutor de ros2)
~~~python
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
import threading
from rclpy.executors import MultiThreadedExecutor
~~~
- Crear una clase que herede de nodo, que implemente un servidor y sus funciones
~~~python
  class Server(Node):
    def __init__(self):
      # Constructor
    def listener_callback(self, msg):
      # Función que se llama al recibir un mensaje
~~~
- Crear una clase que herede de nodo, que implemente un cliente y sus funciones
~~~python
  class Client(Node):
    def __init__(self):
      # Constructor de inicio
    def timer_callback(self):
      # Función de timer para enviar una solicitud periódicamente cada segundo
    def callback_result(self, future:Future):
      # Función que se llama al recibir una respuesta a la solicitud enviada
~~~
- Verificar si ROS2 está inicializado. Si no, inicializarlo. Utilizar la función rclpy.ok(), que nos devuelve el estado de ROS2
~~~python
  if not rclpy.ok():
    rclpy.init(args=None)
~~~
- Instanciar un ejecutor de ROS2 que se encargue de varios nodos de forma simultánea, y ejecutarlo en segundo plano para no bloquear la terminal
~~~ python
  executor = MultiThreadedExecutor()
  thread = threading.Thread(target=executor.spin, daemon=True)
  thread.start()
~~~
- Instanciar los nodos servidor y cliente, y agregarlos al ejecutor
~~~python
  server_node = Server()
  client_node = Client()
  executor.add_node(server_node)
  executor.add_node(client_node)
~~~
- Destruir los nodos para que dejen de ejecutarse
~~~python
  server_node.destroy_node()
  client_node.destroy_node()
~~~
- Que detenga el proceso de ROS2 en caso de que siga activo (para evitar procesos en segundo plano)
~~~python
  if rclpy.ok():
    rclpy.shutdown()
~~~

Como probatorio: 
- Las celdas con código en esta sección del archivo
- Dejar los mensajes de salida que se imprimirán al agregar los nodos al ejecutor

### Código del Nodo Servidor

In [1]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
from example_interfaces.srv import AddTwoInts

class Server(Node):
  def __init__(self):
    super().__init__('server_node')
    server_name = "/mult_two_ints"
    self.srv = self.create_service(AddTwoInts, server_name, self.listener_callback)
    self.get_logger().info('Servicio disponible en {}'.format(server_name))

  def listener_callback(self, request:AddTwoInts.Request, response:AddTwoInts.Response):
    response.sum = request.a * request.b
    self.get_logger().info("Recibido: a={}, b={}, respuesta={}".format(request.a, request.b, response.sum))
    return response

### Código del Nodo Cliente

In [2]:
import rclpy
from rclpy.node import Node
from example_interfaces.srv import AddTwoInts
from rclpy.task import Future

class Client(Node):
  def __init__(self):
    # Constructor de inicio
    super().__init__('client_node')
    service_name = "/mult_two_ints"
    self.a = int(input("Ingrese el primer entero (a): "))
    self.b = int(input("Ingrese el segundo entero (b): "))

    self.client = self.create_client(AddTwoInts, service_name)
  
    while not self.client.wait_for_service(timeout_sec=1.0):
      self.get_logger().info('Esperando al servicio {}...'.format(service_name))
    
    self.timer = self.create_timer(2.0, self.timer_callback)

  def timer_callback(self):
    # Función de timer para enviar solicitudes periódicamente
    self.get_logger().info("Enviando solicitud...")
    request = AddTwoInts.Request()
    request.a = self.a
    request.b = self.b
    future = self.client.call_async(request)
    future.add_done_callback(self.callback_result)

  def callback_result(self, future:Future):
    try:
      response = future.result()
      response:AddTwoInts.Response
      self.get_logger().info("Resultado recibido: {} * {} = {}".format(self.a, self.b, response.sum))
    except Exception as e:
      self.get_logger().error(f"Error al llamar al servicio: {e}")

### Código del Ejecutador de Nodos


In [3]:
import rclpy
from server_node import Server
from client_node import Client
import threading
from rclpy.executors import MultiThreadedExecutor

def initialize_ROS2():
    if not rclpy.ok():
        rclpy.init(args=None)

def instantiate_executor():
    executor = MultiThreadedExecutor()
    thread = threading.Thread(target=executor.spin, daemon=True)
    thread.start()
    # instantiate server and client nodes
    server_node = Server()
    client_node = Client()
    # add nodes to executor
    executor.add_node(server_node)
    executor.add_node(client_node)
    try:
        # keep main thread alive to let executor run
        while rclpy.ok():
            pass
    except KeyboardInterrupt:
        pass
    finally:
      # destroy nodes and shutdown ROS2 when done
      executor.remove_node(server_node)
      executor.remove_node(client_node)
      server_node.destroy_node()
      client_node.destroy_node()
      if rclpy.ok():
          rclpy.shutdown()

def main():
    initialize_ROS2()
    instantiate_executor()

if __name__ == '__main__':
    main()

[INFO] [1762557523.751340947] [server_node]: Servicio disponible en /mult_two_ints
[INFO] [1762557531.363845301] [client_node]: Enviando solicitud...
[INFO] [1762557531.391490099] [server_node]: Recibido: a=6, b=7, respuesta=42
[INFO] [1762557531.423422308] [client_node]: Resultado recibido: 6 * 7 = 42
[INFO] [1762557533.251766571] [client_node]: Enviando solicitud...
[INFO] [1762557533.256595109] [server_node]: Recibido: a=6, b=7, respuesta=42
[INFO] [1762557533.274293618] [client_node]: Resultado recibido: 6 * 7 = 42
[INFO] [1762557535.256967876] [client_node]: Enviando solicitud...
[INFO] [1762557535.272177271] [server_node]: Recibido: a=6, b=7, respuesta=42
[INFO] [1762557535.275253338] [client_node]: Resultado recibido: 6 * 7 = 42
[INFO] [1762557537.252736581] [client_node]: Enviando solicitud...
[INFO] [1762557537.267657121] [server_node]: Recibido: a=6, b=7, respuesta=42
[INFO] [1762557537.312375147] [client_node]: Resultado recibido: 6 * 7 = 42


### Ejecutable en la terminal

<img src = 'imagenes/Ejecutable_2.png'>

### 2. URDF y RViz

En esta sección, se creará un archivo URDF de un robot tipo RRR con la siguiente disposición:
<div align="center">
<img src="imagenes/modelo_robot.png" alt = "Robot RRR" width="300" height="300"/>
</div>

** Se pueden utilizar formas geométricas sencillas
** Mantener las dimensiones menores a 0.5 \[m]. 

#### Despliegue de un modelo URDF con configuraciones predeterminadas en RViz
Ahora, se tomará el archivo URDF realizado y se desplegará en RViz utilizando algunas configuraciones predeterminadas. 
Para desplegar el archivo en RViz, se deben instalar algunas librerías:

Para descargar un paquete que contiene un launch para desplegar al robot
> $ sudo apt-get install ros-humble-urdf-tutorial

Instalar la librería que permite a ROS manipular los URDF (xacro)
> $ sudo apt install ros-humble-xacro

Hay que verificar las juntas del archivo. RViz requiere que las juntas tengan límites definidos, agregando el parámetro
``` xml 
<limit effort="XX" velocity="XX" lower="XX" upper="XX" />
```
Siendo "XX" los valores deseados para cada parámetro.

Con estos cambios, se puede correr el despliegue del modelo con un archivo *.launch del paquete urdf-tutorial 

> $ ros2 launch urdf_tutorial display.launch.py model:=/home/robousr/<ruta_del_modelo>/<nombre_del_modelo>.urdf



Como entregable, agregar una imagen del modelo desplegado en RViz y una celda con el código xml del modelo urdf

<div align="center">
<img src="imagenes/modelo_desplegado.png" alt = "Primer despliegue en RViz" width="1400" height="1000">
</div>

~~~xml
<robot name="rrp_robot"> 
  <!--Definición de eslabones--> 
<link name = "base_link">
  <visual>
      <origin xyz = "0 0 0.05" rpy = "0 0 0"/> 
        <geometry> 
          <cylinder radius="0.2" length="0.1"/> 
        </geometry> 
        <material name = "Black"> 
        <color rgba = "0 0 0 1"/> 
      </material> 
    </visual> 
</link> 
 
<link name = "base2_link"> 
   <visual> 
      <origin xyz = "0 0 0.15" rpy = "0 0 0"/> 
    <geometry> 
      <cylinder radius="0.015" length="0.3"/>
     </geometry> 
      <material name = "Green"> 
        <color rgba = "0 1 0 1.0"/> 
      </material> 
    </visual> 
  </link> 

<link name = "shoulder_link"> 
   <visual> 
      <origin xyz = "0.15 0 0" rpy = "0 0 0"/> 
    <geometry> 
      <box size = "0.3 0.03 0.03"/> 
     </geometry> 
      <material name = "Cyan"> 
        <color rgba = "0 1 1 1.0"/> 
      </material> 
    </visual> 
  </link> 

<link name = "arm_link"> 
   <visual> 
      <origin xyz = "0.15 0 0" rpy = "0 0 0"/> 
    <geometry> 
      <box size = "0.3 0.03 0.03"/> 
     </geometry> 
      <material name = "Magenta"> 
        <color rgba = "1 0 1 1.0"/> 
      </material> 
    </visual> 
  </link> 
<link name = "forearm_link"> 
   <visual> 
      <origin xyz = "0.15 0 0" rpy = "0 0 0"/> 
    <geometry> 
      <box size = "0.3 0.03 0.03"/> 
     </geometry> 
      <material name = "Yellow"> 
        <color rgba = "1 1 0 1.0"/> 
      </material> 
    </visual> 
  </link> 
 
  <!--Definició de juntas--> 
<joint name = "base_joint" type = "fixed"> 
  <parent link = "base_link"/> 
  <child link = "base2_link"/> 
  <origin xyz = "0 0 0.1" rpy = "0 0 0"/> 
</joint>

<joint name = "shoulder_joint" type = "revolute"> 
  <parent link = "base2_link"/> 
  <child link = "shoulder_link"/> 
  <origin xyz = "0 0 0.3" rpy = "0 0 0"/> 
  <axis xyz = "0 0 1"/> 
  <limit effort="10.0" lower="-3.14" upper="3.14" velocity="3.14"/>
</joint>

<joint name = "arm_joint" type = "revolute"> 
  <parent link = "shoulder_link"/> 
  <child link = "arm_link"/> 
  <origin xyz = "0.3 0 0.03" rpy = "0 0 0"/> 
  <axis xyz = "0 0 1"/> 
  <limit effort="10.0" lower="-3.14" upper="3.14" velocity="3.14"/>
</joint>

<joint name = "forearm_joint" type = "prismatic"> 
  <parent link = "arm_link"/> 
  <child link = "forearm_link"/> 
  <origin xyz = "0.3 0 0.15" rpy = "0 1.57 0"/> 
  <axis xyz = "1 0 0"/> 
  <limit effort="10.0" lower="0" upper="0.15" velocity="3.14"/>
</joint>
 
</robot>
~~~

## Análisis de resultados

¿Cuáles son las ventajas o desventajas de usar el protocolo de comunicación publicador/suscriptor ó cliente/servicio?
> RESPUESTA

¿Cuál es la convención de ángulos que utilizan los archivos URDF para los ángulos de las juntas respecto al eslabón padre? (Intrínsecos/extrínsecos y el orden)
> RESPUESTA

¿Qué utilidad tiene describir un robot en un archivo URDF?
> RESPUESTA




## Conclusiones

En esta sección deberan escribir las conclusiones de la práctica con base en el objetivo planteado y las metas que se deban cumplir para su realización.



## Bibliografía 

Hacer referencia a la información implementada en formato ieee, en caso de haberse utilizado